# VacationPy

In [42]:
#List of dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

#Get access to map
from connect import google_string

### _Store WeatherPy results into DataFrame_

In [43]:
#Bring in open weather data from WeatherPy results
cityweather = pd.read_csv(r'../WeatherPy/Results/citydata.csv')
cityweather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,henties bay,-22.1160,14.2845,60.01,95,0,3.00,NaN,1615602849
1,1,bethel,41.3712,-73.4140,53.60,24,1,6.91,US,1615602849
2,2,saint-francois,46.4154,3.9054,45.00,70,75,6.91,FR,1615602850
3,3,ushuaia,-54.8000,-68.3000,46.40,87,75,42.57,AR,1615602850
4,4,mar del plata,-38.0023,-57.5575,64.99,76,0,2.30,AR,1615602850


In [44]:
#Find Humidity Max
hummax= cityweather.Humidity.astype(float).max()
hummax

100.0

## Humidity Heatmap

In [45]:
#Use Googlemaps
gmaps.configure(api_key=google_string)

In [46]:
#Collect Latitudes and Longitudes to plot
locations = cityweather[["Lat","Lng"]]

In [47]:
#Collect Humidity for map weight and convert to float
humidity = cityweather["Humidity"].astype(float)

In [48]:
#Create Heatmap
fig = gmaps.figure(center=[13,4],zoom_level=2)

#Create heat weight layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=hummax, point_radius=3)

#Add heat weight layer
fig.add_layer(heat_layer)

#Display map
fig

Figure(layout=FigureLayout(height='420px'))

### _Weather Criteria_


In [49]:
#Create new data set with weather conditions
condition = cityweather.loc[(cityweather["Max Temp"] < 80) & (cityweather["Max Temp"] > 70) \
                            & (cityweather["Wind Speed"] < 10) \
                            & (cityweather["Cloudiness"] == 0)].dropna()
condition

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,atuona,-9.8000,-139.0333,79.86,58,0,1.48,PF,1615602835
118,118,caravelas,-17.7125,-39.2481,72.75,81,0,5.19,BR,1615602888
137,137,panzhihua,26.5851,101.7128,74.28,28,0,1.07,CN,1615602895
141,141,mecca,21.4267,39.8261,73.24,48,0,5.14,SA,1615602897
193,193,cidreira,-30.1811,-50.2056,70.88,77,0,5.66,BR,1615602917
217,217,belmonte,-15.8631,-38.8828,77.00,65,0,8.05,BR,1615602926
242,242,tecoanapa,16.5167,-98.7500,76.46,79,0,2.44,MX,1615602935
267,267,dourados,-22.2211,-54.8056,71.46,65,0,3.60,BR,1615602944
271,271,talara,-4.5772,-81.2719,78.80,65,0,5.75,PE,1615602946
357,357,bull savanna,17.8868,-77.5902,71.98,79,0,7.58,JM,1615602976


### _Hotel Map_

In [50]:
#Create new data set for hotels with perfect weather conditions
perfecthotels = condition[["City","Country", "Lat", "Lng"]].copy()
perfecthotels["Hotel Name"]=""
perfecthotels

,City,Country,Lat,Lng,Hotel Name
34,atuona,PF,-9.8000,-139.0333,
118,caravelas,BR,-17.7125,-39.2481,
137,panzhihua,CN,26.5851,101.7128,
141,mecca,SA,21.4267,39.8261,
193,cidreira,BR,-30.1811,-50.2056,
217,belmonte,BR,-15.8631,-38.8828,
242,tecoanapa,MX,16.5167,-98.7500,
267,dourados,BR,-22.2211,-54.8056,
271,talara,PE,-4.5772,-81.2719,
357,bull savanna,JM,17.8868,-77.5902,


In [51]:
#Create hotel parameters based on perfect weather conditions
destination = {"radius": 5000, "types": "lodging", "key": google_string}

#Iterate through the cities witht the perfect weather condition
for p, row in perfecthotels.iterrows():
    
    #Find through the latitudes and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    #Use hotel parameters in search
    destination["location"] = f"{lat},{lng}"
    
    #Search url for hotels near cities with perfect weather condition
    googleurl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #Get hotel names
    hotels = requests.get(googleurl, params=destination).json()
    
    #Find and store only one hotel from results
    try:
        perfecthotels.loc[p,"Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        print(f"Hotel Not Found: for {perfecthotels.loc[p,'City']}.")

#New data set
perfecthotels

Hotel Not Found: for koundara.


,City,Country,Lat,Lng,Hotel Name
34,atuona,PF,-9.8000,-139.0333,Villa Enata
118,caravelas,BR,-17.7125,-39.2481,Pousada dos Navegantes
137,panzhihua,CN,26.5851,101.7128,Electric Power Hotel
141,mecca,SA,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
193,cidreira,BR,-30.1811,-50.2056,Hotel Castelo
217,belmonte,BR,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
242,tecoanapa,MX,16.5167,-98.7500,Hotel Manglares
267,dourados,BR,-22.2211,-54.8056,Bravo City Hotel
271,talara,PE,-4.5772,-81.2719,Hostal Oro Negro
357,bull savanna,JM,17.8868,-77.5902,Big Red's


In [52]:
#Filter out cities with no hotel found
vacation = perfecthotels.loc[perfecthotels["Hotel Name"]!= ""]
vacation

,City,Country,Lat,Lng,Hotel Name
34,atuona,PF,-9.8000,-139.0333,Villa Enata
118,caravelas,BR,-17.7125,-39.2481,Pousada dos Navegantes
137,panzhihua,CN,26.5851,101.7128,Electric Power Hotel
141,mecca,SA,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
193,cidreira,BR,-30.1811,-50.2056,Hotel Castelo
217,belmonte,BR,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
242,tecoanapa,MX,16.5167,-98.7500,Hotel Manglares
267,dourados,BR,-22.2211,-54.8056,Bravo City Hotel
271,talara,PE,-4.5772,-81.2719,Hostal Oro Negro
357,bull savanna,JM,17.8868,-77.5902,Big Red's


In [53]:
#Format hotel markers for heatmap
hotel_marks = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

#Data for marker layer rows
hotel_info = [hotel_marks.format(**row) for index, row in vacation.iterrows()]
locations = vacation[["Lat", "Lng"]]

In [54]:
#Placing marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

#Show map
fig

Figure(layout=FigureLayout(height='420px'))